# Sun Access Example


## 0. Initialization

### 0.1. Importing libraries and setting global variables

In [ ]:
import esm
import os
import trimesh as tm 
import ladybug as lb
from honeybee_plus.radiance.recipe.solaraccess.gridbased import SolarAccessGridBased
from honeybee_plus.radiance.analysisgrid import AnalysisGrid

cwd = os.getcwd()
src = os.path.dirname(cwd)

### 0.2. Importing geometries and assigning materials

In [ ]:
# loading the wall mesh
wal_filepath = os.path.join(src, "data", "test_room_walls_02.obj")
wal_mesh = tm.load(wal_filepath)
# converting wall mesh to HB Surfaces
wal_hb_surf = esm.mesh_to_hbsurface(wal_mesh.faces, wal_mesh.vertices, 0, "wall", esm.material_plastic)

# loading the window mesh
fen_filepath = os.path.join(src, "data", "test_room_fens_02.obj")
fen_mesh = tm.load(fen_filepath)
# loading converting window mesh to HB Surfaces
fen_hb_surf = esm.mesh_to_hbsurface(fen_mesh.faces, fen_mesh.vertices, 5, "window", esm.material_glass)

# combining surface lists
hb_surfaces = wal_hb_surf + fen_hb_surf

### 0.3. Prepare Analysis Information

In [ ]:
# generate grid of test points
test_pts = [(1, 1, 1),(6, 1, 1),(1, 6, 1),(1, 1, 6)]
test_vec = [(0, 0, 1),(0, 0, 1),(0, 0, 1),(0, 0, 1)]
analysis_grid = AnalysisGrid.from_points_and_vectors(test_pts, test_vec, 'test_grid')


# initiate sunpath
sp = lb.sunpath.Sunpath(longitude=4.3571, latitude=52.0116)

# define sun hours : A list of hours of the year for each sun vector
# there are 8760 hours in a year, so the following integers refer to specific hours throughout the year
hoys = []
sun_vectors = []
for i in range(8760):
    # compute the sun object
    sun = sp.calculate_sun_from_hoy(i)
    # extract the sun vector
    sun_vector = sun.sun_vector.to_array()
    # apparantly, if the Z component of sun vector is positive, it is under the horizon 
    if sun_vector[2] < 0.0:
        hoys.append(i)
        sun_vectors.append(sun_vector)

## 1. Simulation

### 1.1. Create Recipe

In [ ]:
# put the recipe together
rp = SolarAccessGridBased(sun_vectors, hoys, [analysis_grid], hb_objects=hb_surfaces)

### 1.2. Create Batch Files

In [ ]:
# write simulation to folder
batch_file = rp.write(
    target_folder='.', project_name='sun_acc_cleaned')

### 1.3. Run the simulation

In [ ]:
# run the simulation
rp.run(batch_file, debug=False)

## 2. Results

In [ ]:
result = esm.parse_results(rp, aggregate=False)
print(result)